# Ejercicio 6: Introducción a Dense Retrieval

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), y recuperarlos

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [6]:
from sklearn.datasets import fetch_20newsgroups

# 1. Cargar el corpus completo (usamos subset='all' para incluir todos los documentos)
# Opcionalmente, puedes usar 'remove' para limpiar metadatos como headers, footers y citas.
print("Cargando el corpus completo de 20 Newsgroups...")
corpus_completo = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# 2. Limitar el corpus a los primeros 2000 documentos
LIMITE_DOCUMENTOS = 2000

documentos_limitados = corpus_completo.data[:LIMITE_DOCUMENTOS]
etiquetas_limitadas = corpus_completo.target[:LIMITE_DOCUMENTOS]

print(documentos_limitados[0][:200] + "...")

Cargando el corpus completo de 20 Newsgroups...


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However,...


## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
#all-MiniLM-L6-v2' (SBERT)

MODELO_SBERT = 'all-MiniLM-L6-v2'
print(f"\nCargando y codificando documentos con {MODELO_SBERT}...")

# Carga el modelo SBERT
modelo_sbert = SentenceTransformer(MODELO_SBERT)

# Genera los embeddings. No se necesita prefijo.
# Los resultados son un array de NumPy.
embeddings_sbert = modelo_sbert.encode(
    documentos_limitados,
    show_progress_bar=True,
    convert_to_numpy=True
)

# Guarda los embeddings en un array de NumPy
print(f"Embeddings SBERT generados y guardados en array de NumPy.")
print(f"Forma de los embeddings SBERT: {embeddings_sbert.shape}")


Cargando y codificando documentos con all-MiniLM-L6-v2...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embeddings SBERT generados y guardados en array de NumPy.
Forma de los embeddings SBERT: (2000, 384)


## Parte 3: Consulta
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con similitud coseno.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Definir la consulta
CONSULTA = "car maintenance"
K_TOP = 5 # Número de documentos a recuperar

print(f"Consulta en lenguaje natural: **'{CONSULTA}'**")

#Codificar la consulta
modelo_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Codificar la consulta (no se necesita prefijo para SBERT)
embedding_consulta = modelo_sbert.encode(CONSULTA, convert_to_numpy=True)

# El embedding de la consulta debe ser redimensionado de (384,) a (1, 384)
# para que funcione con cosine_similarity.
embedding_consulta = embedding_consulta.reshape(1, -1)

print(f"Consulta codificada. Forma del embedding: {embedding_consulta.shape}")

Consulta en lenguaje natural: **'car maintenance'**
Consulta codificada. Forma del embedding: (1, 384)


In [9]:
# Calcular la similitud coseno
# Comparamos el embedding de la consulta (1 vector) con todos los embeddings de los documentos (2000 vectores).
similitudes = cosine_similarity(embedding_consulta, embeddings_sbert)

# El resultado es un array de (1, 2000). Lo aplanamos a (2000,)
similitudes = similitudes.flatten()

# Encontrar los índices de los 5 documentos más relevantes
# np.argsort devuelve los índices que ordenarían el array.
# Usamos [::-1] para invertir y obtener los índices de los valores más altos.
indices_top_k = np.argsort(similitudes)[::-1][:K_TOP]

print(f"Similitud coseno calculada. Recuperando los top {K_TOP} documentos.")

Similitud coseno calculada. Recuperando los top 5 documentos.


In [10]:
# 3.1. Mostrar los resultados
print("\n## Documentos más relevantes recuperados")
print("--------------------------------------------------")

for i, index in enumerate(indices_top_k):
    # Obtener el texto del documento original
    documento_recuperado = documentos_limitados[index]
    # Obtener la puntuación de similitud
    similitud_score = similitudes[index]

    # Mostrar la información
    print(f"**TOP {i+1}** (Índice en el Corpus: {index}, Similitud: {similitud_score:.4f})")

    # Mostrar solo los primeros 500 caracteres
    texto_corto = documento_recuperado[:500].replace('\n', ' ') + "..."
    print(f"> {texto_corto}")
    print("---")


## Documentos más relevantes recuperados
--------------------------------------------------
**TOP 1** (Índice en el Corpus: 1822, Similitud: 0.4879)
> As you can see, I have two 1987 cars, both worth about $3000 each. The problem is that maintenance costs on these two cars is running about $4000 per year and insurance $3000 per year.  What am I doing wrong?  Within the last two months, the follows costs have occured:  Dodge 600 SE (Dodge's attempt at the American German car!)  $1,000 - replace head gasket $300   - new radiator  Chevy Nova CL (Chevy's attempt at a Japan import!)  $500 - tune-up,oil change,valve gasket,middle exhaust pipe, misc....
---
**TOP 2** (Índice en el Corpus: 1470, Similitud: 0.4467)
> Archive-name: rec-autos/part1  [most recent changes, 15 March 1993: addition of alt.autos.karting -- rpw]                 === Welcome to Rec.Autos.* ===  This article is sent out automatically each month, and contains a general description of the purpose of each of the automotive 